In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# NonlinearBlockJac

NonlinearBlockJac is a nonlinear solver that uses the block Jacobi method to solve
the system. When to choose this solver over [NonlinearBlockGS](../../../_srcdocs/packages/solvers.nonlinear/nonlinear_block_gs) is an advanced topic, but it is valid for systems that satisfy the same conditions:

1. System (or subsystem) contains a cycle, though subsystems may.
2. System does not contain any implicit states, though subsystems may.

Note that you may not know if you satisfy the second condition, so choosing a solver can be "trial and error." If
NonlinearBlockJac doesn't work, then you will need to use [NewtonSolver](../../../_srcdocs/packages/solvers.nonlinear/newton)

The main difference over `NonlinearBlockGS` is that data passing is delayed until after all subsystems have been
executed.

Here, we choose NonlinearBlockJac to solve the Sellar problem, which has two components with a
cyclic dependency, has no implicit states, and works with Jacobi, although it fails to converge before
hitting the default iteration limit of 10.  In the next section we'll show how to increase the number
of allowed iterations so that it will converge.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src36", get_code("openmdao.test_suite.components.sellar.SellarDis1withDerivatives"), display=False)

:::{Admonition} `SellarDis1withDerivatives` class definition 
:class: dropdown

{glue:}`code_src36`
:::

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src37", get_code("openmdao.test_suite.components.sellar.SellarDis2withDerivatives"), display=False)

:::{Admonition} `SellarDis2withDerivatives` class definition 
:class: dropdown

{glue:}`code_src37`
:::

In [ ]:
import numpy as np
import openmdao.api as om

from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.linear_solver = om.LinearBlockGS()
model.nonlinear_solver = om.NonlinearBlockJac()

prob.setup()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

print(prob['y1'])
print(prob['y2'])

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(prob['y1'], 25.58830273, .00001)
assert_near_equal(prob['y2'], 12.05848819, .00001)

This solver runs all of the subsystems each iteration, but just passes the data along all connections
simultaneously once per iteration. After each iteration, the iteration count and the residual norm are
checked to see if termination has been satisfied.

You can control the termination criteria for the solver using the following options:

## NonlinearBlockJac Options

In [ ]:
om.show_options_table("openmdao.solvers.nonlinear.nonlinear_block_jac.NonlinearBlockJac")

## NonlinearBlockJac Constructor

The call signature for the `NonlinearBlockJac` constructor is:

```{eval-rst}
    .. automethod:: openmdao.solvers.nonlinear.nonlinear_block_jac.NonlinearBlockJac.__init__
        :noindex:
```

## NonlinearBlockJac Option Examples

**maxiter**

  This lets you specify the maximum number of allowed Jacobi iterations during a solve. In this example, we
  increase it from the default, 10, up to 20, so that it successfully converges.

In [ ]:
from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.linear_solver = om.LinearBlockGS()

nlbgs = model.nonlinear_solver = om.NonlinearBlockJac()
nlbgs.options['maxiter'] = 20

prob.setup()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

print(prob['y1'])
print(prob['y2'])

In [ ]:
assert_near_equal(prob['y1'], 25.58830237, .00001)
assert_near_equal(prob['y2'], 12.05848815, .00001)

**atol**

  Here, we set the absolute tolerance to a looser value that will trigger an earlier termination. After
  each iteration, the norm of the residuals is calculated by calling `apply_nonlinear` on all of the components.
  If this norm value is lower than the absolute
  tolerance `atol`, the iteration will terminate.

In [ ]:
from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.linear_solver = om.LinearBlockGS()

nlbgs = model.nonlinear_solver = om.NonlinearBlockJac()
nlbgs.options['atol'] = 1e-2

prob.setup()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

print(prob['y1'])
print(prob['y2'])

In [ ]:
assert_near_equal(prob['y1'], 25.5886171567, .00001)
assert_near_equal(prob['y2'], 12.05848819, .00001)

**rtol**

  Here, we set the relative tolerance to a looser value that will trigger an earlier termination. After
  each iteration, the norm of the residuals is calculated by calling `apply_nonlinear` on all of the components.
  If the ratio of the currently calculated norm to the
  initial residual norm is lower than the relative tolerance `rtol`, the iteration will terminate.

In [ ]:
from openmdao.test_suite.components.sellar import SellarDis1withDerivatives, SellarDis2withDerivatives

prob = om.Problem()
model = prob.model

model.add_subsystem('d1', SellarDis1withDerivatives(), promotes=['x', 'z', 'y1', 'y2'])
model.add_subsystem('d2', SellarDis2withDerivatives(), promotes=['z', 'y1', 'y2'])

model.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                           z=np.array([0.0, 0.0]), x=0.0),
                    promotes=['obj', 'x', 'z', 'y1', 'y2'])

model.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
model.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

model.linear_solver = om.LinearBlockGS()

nlbgs = model.nonlinear_solver = om.NonlinearBlockJac()
nlbgs.options['rtol'] = 1e-3

prob.setup()

prob.set_val('x', 1.)
prob.set_val('z', np.array([5.0, 2.0]))

prob.run_model()

print(prob.get_val('y1'))
print(prob.get_val('y2'))

In [ ]:
assert_near_equal(prob.get_val('y1'), 25.5891491526, .00001)
assert_near_equal(prob.get_val('y2'), 12.0569142166, .00001)